# Resolve Schema

**input**
```
+schema_path: str
```

**methods**
```
+read_json(schema_path: str): dict
+split_json(schema: dict): list
+resolver(entity1: dict, entity2: dict): dict
+resolve_defs(terms: dict, defs: dict) : dict
+ node_order(schema: dict): list
+resolve_nodes(nodeList: list, splitJsonList: list): list
+recombine_nodes(resolvedList: list) : dict
```

In [48]:
import json
from collections import defaultdict, deque
import os
import pandas as pd

class ResolveSchema:
    
    def __init__(self, schema_path: str):
        """
        Initialize the ResolveSchema class.

        Parameters:
        - schema_path (str): The path to the JSON schema file.
        """
        self.schema_path = schema_path
        self.schema = self.read_json(self.schema_path)
        self.nodes = self.get_nodes()
        self.node_pairs = self.get_all_node_pairs()
        self.node_order = self.get_node_order(edges=self.node_pairs)
        self.schema_list = self.split_json()
        self.schema_def = self.return_schema("_definitions.yaml")
        self.schema_term = self.return_schema("_terms.yaml")
        self.schema_def_resolved = self.resolve_references(self.schema_def, self.schema_term)
        self.schema_list_resolved = self.resolve_all_references()
        
    def read_json(self, path: str) -> dict:
        """
        Read a JSON file and return its contents as a dictionary.

        Parameters:
        - path (str): The path to the JSON file.

        Returns:
        - dict: The contents of the JSON file.
        """
        with open(path) as f:
            return json.load(f)
    
    def get_nodes(self) -> list:
        """
        Retrieve all node names from the schema.

        Returns:
        - list: A list of node names.
        """
        nodes = list(self.schema.keys())
        return nodes
    
    def get_node_link(self, node_name: str) -> tuple:
        """
        Retrieve the links and ID for a given node.

        Parameters:
        - node_name (str): The name of the node.

        Returns:
        - tuple: A tuple containing the node ID and its links.
        """
        links = self.schema[node_name]['links']
        node_id = self.schema[node_name]['id']
        if 'subgroup' in links[0]:
            return node_id, links[0]['subgroup']
        else:
            return node_id, links

    def find_upstream_downstream(self, node_name: str) -> list:
        """
        Takes a node name and returns the upstream and downstream nodes.

        Parameters:
        - node_name (str): The name of the node.

        Returns:
        - list: A list of tuples representing upstream and downstream nodes.
        """
        node_id, links = self.get_node_link(node_name)
        
        # Ensure links is a list
        if isinstance(links, dict):
            links = [links]

        results = []

        for link in links:
            target_type = link.get("target_type")
            
            if not node_id or not target_type:
                print("Missing essential keys in link:", link)
                results.append((None, None))
                continue

            results.append((target_type, node_id))

        return results

    def get_all_node_pairs(self, excluded_nodes=["_definitions.yaml", "_terms.yaml", "_settings.yaml", "program.yaml"]) -> list:
        """
        Retrieve all node pairs, excluding specified nodes.

        Parameters:
        - excluded_nodes (list): A list of node names to exclude.

        Returns:
        - list: A list of node pairs.
        """
        node_pairs = []
        for node in self.nodes:
            if not node in excluded_nodes:
                node_pairs.extend(self.find_upstream_downstream(node))
            else:
                continue
        return node_pairs
    
    def get_node_order(self, edges: list) -> list:
        """
        Determine the order of nodes based on their dependencies.

        Parameters:
        - edges (list): A list of tuples representing node dependencies.

        Returns:
        - list: A list of nodes in topological order.
        """
        # Build graph representation
        graph = defaultdict(list)
        in_degree = defaultdict(int)

        for upstream, downstream in edges:
            graph[upstream].append(downstream)
            in_degree[downstream] += 1
            if upstream not in in_degree:
                in_degree[upstream] = 0

        # Perform Topological Sorting (Kahn's Algorithm)
        sorted_order = []
        zero_in_degree = deque([node for node in in_degree if in_degree[node] == 0])

        while zero_in_degree:
            node = zero_in_degree.popleft()
            sorted_order.append(node)
            
            for neighbor in graph[node]:
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    zero_in_degree.append(neighbor)

        # Ensure core_metadata_collection is last
        sorted_order.remove("core_metadata_collection")
        sorted_order.append("core_metadata_collection")

        return sorted_order
    
    def split_json(self) -> list:
        """
        Split the schema into a list of individual node schemas.

        Returns:
        - list: A list of node schemas.
        """
        schema_list = []
        for node in self.nodes:
            schema_list.append(self.schema[node])
        return schema_list
    
    def return_schema(self, target_id: str) -> dict:
        """
        Retrieves the first dictionary from a list where the 'id' key matches the target_id.

        Parameters:
        - target_id (str): The value of the 'id' key to match.

        Returns:
        - dict: The dictionary that matches the target_id, or None if not found.
        """
        if target_id.endswith('.yaml'):
            target_id = target_id[:-5]
        
        result = next((item for item in self.schema_list if item.get('id') == target_id), None)
        if result is None:
            print(f"{target_id} not found")
        return result
    
    def resolve_references(self, schema: dict, reference: dict) -> dict:
        """
        Takes a gen3 jsonschema draft 4 as a dictionary and recursively resolves any references using a reference schema which has no references.

        Parameters:
        - schema (dict): The JSON node to resolve references in.
        - reference (dict): The schema containing the references.

        Returns:
        - dict: The resolved JSON node with references resolved.
        """
        ref_input_content = reference

        def resolve_node(node, manual_ref_content=ref_input_content):
            if isinstance(node, dict):
                if '$ref' in node:
                    ref_path = node['$ref']
                    ref_file, ref_key = ref_path.split('#')
                    ref_file = ref_file.strip()
                    ref_key = ref_key.strip('/')
                
                    # if a reference file is in the reference, load the pre-defined reference, if no file exists, then use the schema itself as reference
                    if ref_file:
                        ref_content = manual_ref_content
                    else:
                        ref_content = schema
                    
                    for part in ref_key.split('/'):
                        ref_content = ref_content[part]

                    resolved_content = resolve_node(ref_content)
                    # Merge resolved content with the current node, excluding the $ref key
                    return {**resolved_content, **{k: resolve_node(v) for k, v in node.items() if k != '$ref'}}
                else:
                    return {k: resolve_node(v) for k, v in node.items()}
            elif isinstance(node, list):
                return [resolve_node(item) for item in node]
            else:
                return node

        return resolve_node(schema)
    
    def resolve_all_references(self) -> list:
        """
        Resolves references in all other schema dictionaries using the resolved definitions schema.

        Returns:
        - list: A list of resolved schema dictionaries.
        """
        resolved_schema_list = []
        for node in self.nodes:
            
            if node == "_definitions.yaml" or node == "_terms.yaml":
                continue
            
            try:
                resolved_schema = self.resolve_references(self.schema[node], self.schema_def_resolved)
                resolved_schema_list.append(resolved_schema)
                print(f"Resolved {node}")
            except KeyError as e:
                print(f"Error resolving {node}: Missing key {e}")
            except Exception as e:
                print(f"Error resolving {node}: {e}")
                
        return resolved_schema_list
    
    
    def return_resolved_schema(self, target_id: str) -> dict:
        """
        Retrieves the first dictionary from a list where the 'id' key matches the target_id.

        Parameters:
        - target_id (str): The value of the 'id' key to match.

        Returns:
        - dict: The dictionary that matches the target_id, or None if not found.
        """
        if target_id.endswith('.yaml'):
            target_id = target_id[:-5]
        
        result = next((item for item in self.schema_list_resolved if item.get('id') == target_id), None)
        if result is None:
            print(f"{target_id} not found")
        return result
    
    
    
#### Parse Data Class

class ParseData():
    """Parses a json data into a list of dictionaries"""
    
    def __init__(self, data_folder_path: str = None, data_file_path: str = None):
        self.folder_path = data_folder_path
        self.file_path = data_file_path
        self.file_path_list = self.list_data_files()
        self.data_list = self.load_json_data(self.file_path_list)
        self.data_nodes = self.get_node_names()
        
        
        
    def read_json(self, path: str) -> dict:
        with open(path) as f:
            data = json.load(f)
            return data
    
    
    def list_data_files(self) -> list:
        """
        Lists all JSON data files in the specified folder or returns the single file path.

        This method checks if a folder path is provided. If so, it lists all files in the folder
        that have a '.json' extension and returns their absolute paths. If no folder path is provided,
        it returns the single file path specified during initialization.

        Returns:
        - list: A list of absolute file paths to JSON files.
        """
        if self.folder_path:
            json_paths = [os.path.abspath(os.path.join(self.folder_path, f)) for f in os.listdir(self.folder_path) if f.endswith(".json")]
        else:
            json_paths = [self.file_path]
        return json_paths
        
        
    def load_json_data(self, json_paths: list) -> list:
        """
        Loads JSON data from a list of file paths.

        This method reads each JSON file specified in the json_paths list,
        loads the data, and appends it to a list of JSON objects.

        Parameters:
        - json_paths (list): A list of file paths to JSON files.

        Returns:
        - list: A list of dictionaries containing the data from each JSON file.
        """
        json_files = []
        for file in json_paths:
            json_data = self.read_json(file)
            json_files.append(json_data)
            print(f"Loaded {file}")
        
        return json_files
    
    
    def get_node_names(self) -> list:
        """
        Retrieves the names of nodes from the JSON files.

        This method iterates over the list of file paths and extracts the node names
        by removing the '.json' extension from each file name.

        Returns:
        - list: A list of node names extracted from the JSON file paths.
        """
        node_names = []
        for node in self.file_path_list:
            if node.endswith('.json'):
                last_item = os.path.basename(node)
                node_names.append(last_item[:-5])
            else:
                node_names.append(node)
        return node_names


class ParseXlsxMetadata():
    def __init__(self, xlsx_path: str):
        self.xlsx_path = xlsx_path
        self.xlsx_data_dict = self.parse_metadata_template()
        self.sheet_names = self.get_sheet_names()
        
    def parse_metadata_template(self) -> dict:
        """
        Parses an Excel file and converts each sheet into a DataFrame.

        This function reads an Excel file specified by the `xlsx_path` and loads each sheet
        into a dictionary where the keys are the sheet names and the values are the DataFrames
        representing the data in those sheets. The first row of each DataFrame is removed.

        Args:
        - xlsx_path (str): The path to the Excel file to be parsed.

        Returns:
        - dict: A dictionary where each key is a sheet name and each value is a DataFrame
        containing the data from that sheet, with the first row removed.
        """
        # load xlsx file
        pd_dict = pd.read_excel(self.xlsx_path, sheet_name=None)

        # in each pandas data fram in the dict, remove the first row
        for key in pd_dict.keys():
            pd_dict[key] = pd_dict[key].iloc[1:, :]

        return pd_dict
    
    
    def get_sheet_names(self) -> list:
        return list(self.xlsx_data_dict.keys())
    
    
    def get_pk_fk_pairs(self, sheet_name: str) -> tuple:
        """
        Extracts the primary key (PK) and foreign key (FK) column names from a specified sheet.

        This method retrieves the first two column names from the given sheet in the Excel data dictionary,
        assuming the first column is the primary key and the second column is the foreign key.

        Args:
        - sheet_name (str): The name of the sheet from which to extract the PK and FK.

        Returns:
        - tuple: A tuple containing the primary key and foreign key column names.
        """
        sheet = self.xlsx_data_dict[sheet_name]
        first_two_columns = sheet.columns[:2].tolist()
        pk = first_two_columns[0]
        fk = first_two_columns[1]
        return pk, fk
    
    
    def pd_to_json(self, sheet_name: str, json_path: str) -> None:
        """
        Converts a specified sheet from the metadata dictionary to a JSON file. 
        Also formats and renames the primary and foreign keys into a gen3 compatible format

        Args:
        - metadata (dict): A dictionary where each key is a sheet name and each value is a DataFrame.
        - sheet_name (str): The name of the sheet to convert to JSON.
        - json_path (str): The path to the JSON file to be saved.

        Returns:
        - None
        """
        
        pk, fk = self.get_pk_fk_pairs(sheet_name)

        df = self.xlsx_data_dict[sheet_name]
        df['type'] = sheet_name # add node / entity name
        fk_name = fk.split('_uid')[0]
        df[f"{fk_name}s"] = df[fk].apply(lambda x: {"submitter_id": x}) # format foreign key
        df_cleaned = df.where(pd.notnull(df), None)
        df_cleaned['submitter_id'] = df_cleaned[pk].tolist() # adding primary key to submitter_id key
        df_cleaned = df_cleaned.loc[:, ~df_cleaned.columns.str.endswith('_uid')]
        data_list = df_cleaned.to_dict(orient='records')
        

        with open(json_path, 'w') as f:
            json.dump(data_list, f)

        
class TestLinkage(ResolveSchema, ParseData):
    def __init__(self, schema_path, data_folder_path: str = None, data_file_path: str = None):
        super().__init__(schema_path)
        self.data_inst = ParseData(data_folder_path, data_file_path)
        self.data_list = self.data_inst.data_list
        self.data_nodes = self.data_inst.data_nodes
        
    # def pull_keys(self, suffix: str = '_uid'):
        
    
    
    # def find_pk(self, suffix: str = '_uid')
        
        
        
        

In [49]:
metadata_inst = ParseXlsxMetadata('../data/lipid_metadata_example.xlsx')
metadata_sheets = metadata_inst.sheet_names

for sheet in metadata_sheets:
    metadata_inst.pd_to_json(sheet, f'../data/lipid_pass/{sheet}.json')

In [ ]:
# Loading and saving xlsx to individual jsons based on tabs
import pandas as pd


def parse_metadata_template(xlsx_path: str) -> dict:
    """
    Parses an Excel file and converts each sheet into a DataFrame.

    This function reads an Excel file specified by the `xlsx_path` and loads each sheet
    into a dictionary where the keys are the sheet names and the values are the DataFrames
    representing the data in those sheets. The first row of each DataFrame is removed.

    Args:
    - xlsx_path (str): The path to the Excel file to be parsed.

    Returns:
    - dict: A dictionary where each key is a sheet name and each value is a DataFrame
      containing the data from that sheet, with the first row removed.
    """
    # load xlsx file
    pd_dict = pd.read_excel(xlsx_path, sheet_name=None)

    # in each pandas data fram in the dict, remove the first row
    for key in pd_dict.keys():
        pd_dict[key] = pd_dict[key].iloc[1:, :]

    return pd_dict

metadata = parse_metadata_template('../data/lipid_metadata.xlsx')


def xlsx_to_json(xlsx_path: str, json_path: str) -> None:
    """
    Converts an Excel file to a JSON file.

    This function reads an Excel file specified by the `xlsx_path` and loads each sheet
    into a dictionary where the keys are the sheet names and the values are the DataFrames
    representing the data in those sheets. The first row of each DataFrame is removed.

    Args:
    - xlsx_path (str): The path to the Excel file to be parsed.
    - json_path (str): The path to the JSON file to be saved.

    Returns:
    - None
    """
    pd_dict = parse_metadata_template(xlsx_path)
    with open(json_path, 'w') as f:
        json.dump(pd_dict, f)
        

In [17]:
import json

def pd_to_json(metadata: dict, sheet_name: str, json_path: str) -> None:
    """
    Converts a specified sheet from the metadata dictionary to a JSON file.

    Args:
    - metadata (dict): A dictionary where each key is a sheet name and each value is a DataFrame.
    - sheet_name (str): The name of the sheet to convert to JSON.
    - json_path (str): The path to the JSON file to be saved.

    Returns:
    - None
    """
    data_list = metadata[sheet_name].reset_index().rename(columns={'index': 'id'}).to_dict(orient='records')
    with open(json_path, 'w') as f:
        json.dump(data_list, f)

# Example usage
pd_to_json(metadata, 'lipidomics_mapping_file', '../data/lipidomics_mapping_file.json')

In [47]:
linkage_class = TestLinkage(schema_path = "../schema/gen3_test_schema.json", data_folder_path = "../data/pass")


Resolved demographic.yaml
Resolved project.yaml
Resolved serum_marker_assay.yaml
Resolved alignment_workflow.yaml
Resolved imaging_file.yaml
Resolved lipidomics_assay.yaml
Resolved metabolomics_file.yaml
Resolved acknowledgement.yaml
Resolved medical_history.yaml
Resolved _settings.yaml
Resolved blood_pressure_test.yaml
Resolved genomics_assay.yaml
Resolved variant_file.yaml
Resolved program.yaml
Resolved serum_marker_file.yaml
Resolved proteomics_assay.yaml
Resolved sample.yaml
Resolved unaligned_reads_file.yaml
Resolved aligned_reads_index_file.yaml
Resolved variant_workflow.yaml
Resolved proteomics_file.yaml
Resolved exposure.yaml
Resolved metabolomics_assay.yaml
Resolved lipidomics_mapping_file.yaml
Resolved lipidomics_file.yaml
Resolved aligned_reads_file.yaml
Resolved lab_result.yaml
Resolved medication.yaml
Resolved publication.yaml
Resolved subject.yaml
Resolved core_metadata_collection.yaml
Loaded /Users/harrijh/projects/gen3-data-validator/data/pass/metabolomics_file.json
Loa

['metabolomics_file',
 'medical_history',
 'metabolomics_assay',
 'sample',
 'subject']

In [26]:
data = ParseData(data_folder_path = "../data/pass")

Loaded /Users/harrijh/projects/gen3-data-validator/data/pass/metabolomics_file.json
Loaded /Users/harrijh/projects/gen3-data-validator/data/pass/medical_history.json
Loaded /Users/harrijh/projects/gen3-data-validator/data/pass/metabolomics_assay.json
Loaded /Users/harrijh/projects/gen3-data-validator/data/pass/sample.json
Loaded /Users/harrijh/projects/gen3-data-validator/data/pass/subject.json


In [28]:
data.get_node_names()

['metabolomics_file',
 'medical_history',
 'metabolomics_assay',
 'sample',
 'subject']

In [8]:
resolved_schema = ResolveSchema("../schema/gen3_test_schema.json")
resolved_schema.return_resolved_schema('subject')

Resolved demographic.yaml
Resolved project.yaml
Resolved serum_marker_assay.yaml
Resolved alignment_workflow.yaml
Resolved imaging_file.yaml
Resolved lipidomics_assay.yaml
Resolved metabolomics_file.yaml
Resolved acknowledgement.yaml
Resolved medical_history.yaml
Resolved _settings.yaml
Resolved blood_pressure_test.yaml
Resolved genomics_assay.yaml
Resolved variant_file.yaml
Resolved program.yaml
Resolved serum_marker_file.yaml
Resolved proteomics_assay.yaml
Resolved sample.yaml
Resolved unaligned_reads_file.yaml
Resolved aligned_reads_index_file.yaml
Resolved variant_workflow.yaml
Resolved proteomics_file.yaml
Resolved exposure.yaml
Resolved metabolomics_assay.yaml
Resolved lipidomics_mapping_file.yaml
Resolved lipidomics_file.yaml
Resolved aligned_reads_file.yaml
Resolved lab_result.yaml
Resolved medication.yaml
Resolved publication.yaml
Resolved subject.yaml
Resolved core_metadata_collection.yaml


{'$schema': 'http://json-schema.org/draft-04/schema#',
 'additionalProperties': False,
 'category': 'clinical',
 'description': 'An individual participant in the study with baseline measurements.',
 'id': 'subject',
 'links': [{'backref': 'subjects',
   'label': 'part_of',
   'multiplicity': 'many_to_one',
   'name': 'projects',
   'required': True,
   'target_type': 'project'}],
 'namespace': 'https://data.test.biocommons.org.au/',
 'program': '*',
 'project': '*',
 'properties': {'created_datetime': {'oneOf': [{'format': 'date-time',
     'type': 'string'},
    {'type': 'null'}],
   'term': {'description': 'A combination of date and time of day in the form [-]CCYY-MM-DDThh:mm:ss[Z|(+|-)hh:mm]\n'}},
  'id': {'pattern': '^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$',
   'term': {'description': 'A 128-bit identifier. Depending on the mechanism used to generate it, it is either guaranteed to be different from all other UUIDs/GUIDs generated until 3400 AD 

In [13]:
resolved_schema = ResolveSchema("../schema/gen3_test_schema.json")
def_schema = resolved_schema.return_schema('_definitions.yaml')
terms_schema = resolved_schema.return_schema('_terms.yaml')
sample_schema = resolved_schema.return_schema('sample.yaml')
demographic_schema = resolved_schema.return_schema('demographic.yaml')


def resolve_references(schema: dict, reference: dict) -> dict:
    """
    Takes a gen3 jsonschema draft 4 as a dictionary and recursively resolves any references using a reference schema which has no references.

    Parameters:
    - schema (dict): The JSON node to resolve references in.
    - reference (dict): the schema containing the references

    Returns:
    - dict: The resolved JSON node with references resolved.
    """
    
    ref_input_content = reference


    def resolve_node(node, manual_ref_content=ref_input_content):
        if isinstance(node, dict):
            if '$ref' in node:
                ref_path = node['$ref']
                ref_file, ref_key = ref_path.split('#')
                ref_file = ref_file.strip()
                ref_key = ref_key.strip('/')
                # print(f'Resolving $ref: {ref_file}#{ref_key}')
            
                # if a reference file is in the reference, load the pre-defined reference, if no file exists, then use the schema itself as reference
                if ref_file:
                    ref_content = manual_ref_content
                else:
                    ref_content = schema
                
                
                for part in ref_key.split('/'):
                    ref_content = ref_content[part]

                resolved_content = resolve_node(ref_content)
                # Merge resolved content with the current node, excluding the $ref key
                return {**resolved_content, **{k: resolve_node(v) for k, v in node.items() if k != '$ref'}}
            else:
                return {k: resolve_node(v) for k, v in node.items()}
        elif isinstance(node, list):
            return [resolve_node(item) for item in node]
        else:
            return node

    return resolve_node(schema)

def_resolved = resolve_references(def_schema, terms_schema)
def_resolved
resolve_references(sample_schema, def_resolved)

{'$schema': 'http://json-schema.org/draft-04/schema#',
 'additionalProperties': False,
 'category': 'biospecimen',
 'description': "Biospecimen information that links subjects to samples including sample's provider and source.",
 'id': 'sample',
 'links': [{'backref': 'samples',
   'label': 'taken_from',
   'multiplicity': 'many_to_one',
   'name': 'subjects',
   'required': True,
   'target_type': 'subject'}],
 'namespace': 'https://data.test.biocommons.org.au/',
 'program': '*',
 'project': '*',
 'properties': {'created_datetime': {'oneOf': [{'format': 'date-time',
     'type': 'string'},
    {'type': 'null'}],
   'term': {'description': 'A combination of date and time of day in the form [-]CCYY-MM-DDThh:mm:ss[Z|(+|-)hh:mm]\n'}},
  'id': {'pattern': '^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$',
   'term': {'description': 'A 128-bit identifier. Depending on the mechanism used to generate it, it is either guaranteed to be different from all other UUID

In [ ]:
ref_dict = {
    resolved_schema.schema['demographic.yaml']['links']: resolved_schema.schema['demographic.yaml']['links']
    }


In [22]:
resolved_schema.get_node_link('lipidomics_file.yaml')

[{'backref': 'lipidomics_files',
  'label': 'data_from',
  'multiplicity': 'many_to_one',
  'name': 'lipidomics_assays',
  'required': False,
  'target_type': 'lipidomics_assay'},
 {'backref': 'lipidomics_files',
  'label': 'data_from',
  'multiplicity': 'one_to_one',
  'name': 'core_metadata_collections',
  'required': False,
  'target_type': 'core_metadata_collection'}]

In [23]:
# for getting upstream downstream from links
def find_upstream_downstream(links):
    """Takes a list of dictionaries or a single dictionary of links and returns the upstream and downstream nodes"""

    # Ensure links is a list
    if isinstance(links, dict):
        links = [links]

    results = []

    for link in links:
        backref = link.get("backref")
        target_type = link.get("target_type")
        
        if not backref or not target_type:
            print("Missing essential keys in link:", link)
            results.append((None, None))
            continue
        
        # strip last s from name
        if backref.endswith('s'):
            backref = backref[:-1]

        # Determine upstream/downstream logic
        upstream = target_type
        downstream = backref
        
        print(f"Upstream: {upstream}, Downstream: {downstream}")

        results.append((upstream, downstream))

    return results

In [25]:
link = resolved_schema.get_node_link('lipidomics_file.yaml')
find_upstream_downstream(link)

Upstream: lipidomics_assay, Downstream: lipidomics_file
Upstream: core_metadata_collection, Downstream: lipidomics_file


[('lipidomics_assay', 'lipidomics_file'),
 ('core_metadata_collection', 'lipidomics_file')]